In [1]:
import pandas as pd
from selenium.webdriver import Firefox
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
df = pd.read_excel("CREA-MS.xlsx", sheet_name=0, nrows=30)
df = df.drop(columns=['A','B', 'RT'], axis=1)
df2 = df[df['Situação'] == 'ATIVA']
df2

,Empresa,Registro,Situação
0,A7 AVIAÇÃO AGRÍCOLA,MS374,ATIVA
1,IMAGETECH TECNOLOGIA EM INFORMÁTICA LTDA,MS7507,ATIVA
2,03 IRMÃOS PRESTADORA DE SERVIÇOS,MS18358,ATIVA
3,1 PRA 2 ARQUITETURA E ENGENHARIA,MS19107,ATIVA
5,1A SERVICE,MS20561,ATIVA
6,2 A AVIAÇÃO AGRICOLA,MS18912,ATIVA
7,2.0 ENGENHARIA,MS19816,ATIVA
9,2J INCORPORACAO E EMPRENDIMENTOS IMOBILIARIOS ...,MS18939,ATIVA
10,2M INSTALAÇÃO E MANUTENÇÃO,MS10822,ATIVA
11,2P ARQUITETURA E ENG,MS4261,ATIVA


In [3]:
i=0
busca = df2.iloc[i]['Empresa']
busca

'A7 AVIAÇÃO AGRÍCOLA'

In [4]:
listacnpj =[]

In [5]:
def scrapiando(busca):
    url = 'http://cnpj.info/'

    cnpj = []
    nome = []
    fantasia = []
    estado = []
    limite_requisicoes = 0
    
    browser = Firefox()
    browser.get(url)
    
    divclass = browser.find_element_by_class_name('hdr')
    element = divclass.find_element_by_tag_name('h3')
    insert = element.find_element_by_name('q')
    click = element.find_element_by_xpath("//input[@title='Search/Lookup']")

# Insere o texto com o nome da empresa na caixa de busca do navegador
    insert.send_keys(busca)

# Clica no botão Buscar, outra opção é teclar enter no campo de busca = insert.send_keys(Keys.RETURN)
    click.click()

# Atualiza o browser para a pagina atual, a pagina nova carregada após a busca
    browser.current_window_handle
    
    listas = browser.find_elements_by_tag_name('li')

    text_site = 'Too many requests per address per day... try to access this site later.'
    text_many = browser.find_element_by_tag_name('h1')
    if text_many.text == text_site:
        print("Muitas requisições feitas hoje, bora mudar o IP do proxy")
        browser.quit()
        return 5

    if len(listas) == 0:
        browser.quit()
        return   
    
    i = 0
    while i < len(listas):
        print(listas[i].text +'\n')
        i = i+1
        
    i = 0
    while i < len(listas):
        inside = listas[i].find_elements_by_tag_name('a')
    
        try:
            cnpj.append(inside[0].text)
        except IndexError:
            cnpj.append('NaN')

        try:
            nome.append(inside[1].text)
        except IndexError:
            nome.append('NaN')    
    
        try:
            fantasia.append(inside[2].text)
        except IndexError:
            fantasia.append('NaN')
    
        # Extrai o Estado do país contído no texto
        list2 = listas[i].text
        list3 = list2.splitlines()
        state = list3[-1][-2:] # Pega 2 últimas letras do último item da lista. Exemplo: 'Localização: Centro, Porto Grande - AP'
        state = state.strip()
        estado.append(state)
    
        i = i+1
        
    print(cnpj)
    print(nome)
    print(fantasia)
    print(estado)
    
    dicio = {"CNPJ":cnpj, "Nome":nome, "Fantasia": fantasia, "Estado": estado}
    df= pd.DataFrame(dicio)

    dfnew = df[df['Estado'] == 'MS']
    
    if len(dfnew) == 0:
        browser.quit()
        return
    
    listacnpj.append(dfnew.iloc[0]['CNPJ'])
    
    browser.quit()

In [ ]:
for j in range(len(df2)):
    procura = df.iloc[j]['Empresa']
    
    scrapiando(procura)
          
    if scrapiando(procura) == 5:
        print("Encerrando busca por agora")
        break

Muitas requisições feitas hoje, bora mudar o IP do proxy


In [ ]:
print(listacnpj)